In [38]:
import pandas as pd
import numpy as np

In [17]:
#pd.set_option('display.max_rows', None)

In [48]:
#pd.reset_option('display.max_rows')

In [10]:
d = pd.read_csv('data/tokens.txt', header=None, delimiter=' ')

In [31]:
#d.to_pickle('data/tokens.pkl')

In [5]:
#d = pd.read_pickle('data/tokens.pkl')

In [30]:
# print(d.iloc[:30, :3])

                                                    0         1         2
0                                                 the  0.306717 -0.320530
1                                                   ,  0.022532  0.320788
2                                                   . -0.075626  0.369955
3                                                  of  0.487177 -0.193218
4                                                  to  0.173940 -0.104660
5                                                 and  0.222883 -0.001161
6                                                  in  0.712315  0.451855
7                                                   a  0.048874  0.379618
8    0.019833000000000017 0.13675199999999998 -0.4... -0.246612  0.579491
9                                               1,028  0.173451  0.174839
10                                                12m  0.296672 -0.595140
11                                               2037 -0.612709 -0.547751
12                                    

In [45]:
first_ten_words = d.iloc[[0,3,4,5,6,7,15,16,17,18]]

In [96]:
#cosine similarity = v1 dot v2 / (norm(v1) * norm(v2))
#cosine distance = 1 - cosine similarity

cos_similarities = np.empty((10,10))
for i in range(10):
    for j in range (10):
        cos_similarities[i, j] = np.dot(first_ten_words.iloc[i, 1:], first_ten_words.iloc[j, 1:]) / (np.linalg.norm(first_ten_words.iloc[i, 1:]) * np.linalg.norm(first_ten_words.iloc[j, 1:]))
cos_similarities = np.clip(cos_similarities, -1, 1) #clipped to fix a rounding error
cos_distances = 1 - cos_similarities

print(np.round(cos_distances, decimals=3))

[[0.    0.145 0.199 0.205 0.153 0.18  1.118 1.077 1.13  1.088]
 [0.145 0.    0.271 0.191 0.173 0.209 1.068 1.052 1.083 1.154]
 [0.199 0.271 0.    0.194 0.218 0.25  1.069 1.059 1.134 1.255]
 [0.205 0.191 0.194 0.    0.172 0.262 1.109 1.063 1.084 1.143]
 [0.153 0.173 0.218 0.172 0.    0.189 1.097 1.113 1.143 1.106]
 [0.18  0.209 0.25  0.262 0.189 0.    1.095 1.079 1.156 1.102]
 [1.118 1.068 1.069 1.109 1.097 1.095 0.    0.796 1.026 0.884]
 [1.077 1.052 1.059 1.063 1.113 1.079 0.796 0.    0.861 0.841]
 [1.13  1.083 1.134 1.084 1.143 1.156 1.026 0.861 0.    0.905]
 [1.088 1.154 1.255 1.143 1.106 1.102 0.884 0.841 0.905 0.   ]]


In [98]:
#angular distance = arccos(v1 dot v2 / (norm(v1) * norm(v2)))

angular_distances = np.arccos(cos_similarities)

print(np.round(angular_distances, decimals=3))

[[0.    0.545 0.642 0.652 0.56  0.61  1.689 1.648 1.701 1.659]
 [0.545 0.    0.753 0.628 0.598 0.658 1.639 1.622 1.654 1.726]
 [0.642 0.753 0.    0.633 0.673 0.723 1.64  1.63  1.705 1.828]
 [0.652 0.628 0.633 0.    0.595 0.741 1.68  1.634 1.655 1.714]
 [0.56  0.598 0.673 0.595 0.    0.626 1.668 1.684 1.714 1.677]
 [0.61  0.658 0.723 0.741 0.626 0.    1.666 1.65  1.727 1.673]
 [1.689 1.639 1.64  1.68  1.668 1.666 0.    1.366 1.597 1.455]
 [1.648 1.622 1.63  1.634 1.684 1.65  1.366 0.    1.431 1.411]
 [1.701 1.654 1.705 1.655 1.714 1.727 1.597 1.431 0.    1.476]
 [1.659 1.726 1.828 1.714 1.677 1.673 1.455 1.411 1.476 0.   ]]


In [124]:
ang_dis_arr_rounded_no_repeats = np.tril(np.round(angular_distances, decimals=4))
ang_dis_df = pd.DataFrame(columns=['word1', 'word2', 'angular distance'])
for i in range(10):
    for j in range(10):
        if ang_dis_arr_rounded_no_repeats[i,j] != 0:
            ang_dis_df.loc[len(ang_dis_df)] = {'word1': first_ten_words.iloc[i,0], 'word2': first_ten_words.iloc[j,0], 'angular distance': ang_dis_arr_rounded_no_repeats[i,j]}

ang_dis_dif = ang_dis_df.sort_values('angular distance')
print(ang_dis_dif)

           word1         word2  angular distance
0             of           the            0.5455
6             in           the            0.5601
9             in           and            0.5945
7             in            of            0.5979
10             a           the            0.6097
14             a            in            0.6257
4            and            of            0.6276
5            and            to            0.6329
1             to           the            0.6416
3            and           the            0.6518
11             a            of            0.6582
8             in            to            0.6727
12             a            to            0.7225
13             a           and            0.7407
2             to            of            0.7534
27    bafflement  acetaldehyde            1.3659
43     bookended    bafflement            1.4108
35       berhanu    bafflement            1.4315
42     bookended  acetaldehyde            1.4547
44     bookended    

In [138]:
#for comparison - the list is the same with both distances measures

cos_dis_arr_rounded_no_repeats = np.tril(np.round(cos_distances, decimals=4))
cos_dis_df = pd.DataFrame(columns=['word1', 'word2', 'cosine distance'])
for i in range(10):
    for j in range(10):
        if cos_dis_arr_rounded_no_repeats[i,j] != 0:
            cos_dis_df.loc[len(cos_dis_df)] = {'word1': first_ten_words.iloc[i,0], 'word2': first_ten_words.iloc[j,0], 'cosine distance': cos_dis_arr_rounded_no_repeats[i,j]}

cos_dis_dif = cos_dis_df.sort_values('cosine distance')
#print(cos_dis_dif)